**Упрощенный семантический анализатор**  
**частоты повторений слов и фраз на основе выборки по ключевым словам из массива данных.  
v.1.2**  


Для свободного использования

Автор: Игорь Лубенко   
https://github.com/IgorLubenko  
lubenkoiv@yandex.ru

**Внимание! Важная информация для использования:**

***1. Возможно использование данный парсер в двух вариантах:***  
а) Полный: когда на вход подается файл с необработанными данными (например, data_jobs.csv) и алгорим выполняется с первого шага и до конца. В этом случае выполнение раздела 5 отключено (задокументировано);  
б) Усеченный, когда на вход подается файл 'jobs_qa.csv' с ранее отфильтрованными данными, по которым затем выполняется семантический анализ. В этом случае шаги 2 - 4 следует пропустить и после выполнения шага 1 продолжить с шага №5;  


***2. В алгоритме две ключевые развилки:***  
  - выбор ключевых слов для отбора из массива данных - производится в разделе 4;  
  - выбор колонки - источника для дальнейшего анализа количества повторяемых слов и фраз: производится в разделе 8

<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-библиотек-и-начальные-операции" data-toc-modified-id="Загрузка-библиотек-и-начальные-операции-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка библиотек и начальные операции</a></span></li><li><span><a href="#Загрузка-начальных-данных" data-toc-modified-id="Загрузка-начальных-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка начальных данных</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#ВНИМАНИЕ:-ключевой-раздел!-Создание-выборки-по-ключевым-словам" data-toc-modified-id="ВНИМАНИЕ:-ключевой-раздел!-Создание-выборки-по-ключевым-словам-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>ВНИМАНИЕ: ключевой раздел! Создание выборки по ключевым словам</a></span></li><li><span><a href="#Загрузка-отфильтрованных-данных-(короткий-вариант,-если-фильтрация-по-словам-уже-выполнена)" data-toc-modified-id="Загрузка-отфильтрованных-данных-(короткий-вариант,-если-фильтрация-по-словам-уже-выполнена)-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Загрузка отфильтрованных данных (короткий вариант, если фильтрация по словам уже выполнена)</a></span></li><li><span><a href="#Исследование-данных" data-toc-modified-id="Исследование-данных-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Исследование данных</a></span></li><li><span><a href="#Очистка-от-дубликатов" data-toc-modified-id="Очистка-от-дубликатов-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Очистка от дубликатов</a></span></li><li><span><a href="#Формирование-данных-с-требованиями-к-кандидатам" data-toc-modified-id="Формирование-данных-с-требованиями-к-кандидатам-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Формирование данных с требованиями к кандидатам</a></span></li><li><span><a href="#Очистка-данных:-удаление-ненужных-знаков-и-символов" data-toc-modified-id="Очистка-данных:-удаление-ненужных-знаков-и-символов-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Очистка данных: удаление ненужных знаков и символов</a></span></li><li><span><a href="#Подсчет-количества-повторяющихся-слов" data-toc-modified-id="Подсчет-количества-повторяющихся-слов-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Подсчет количества повторяющихся слов</a></span></li><li><span><a href="#Выделение-единичных-уникальных-слов-(токенизация-по-словам)" data-toc-modified-id="Выделение-единичных-уникальных-слов-(токенизация-по-словам)-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Выделение единичных уникальных слов (токенизация по словам)</a></span></li><li><span><a href="#Нахождение-словосочетаний-(n-grams)" data-toc-modified-id="Нахождение-словосочетаний-(n-grams)-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Нахождение словосочетаний (n-grams)</a></span></li><li><span><a href="#Вывод-результата-на-экран" data-toc-modified-id="Вывод-результата-на-экран-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Вывод результата на экран</a></span></li><li><span><a href="#Результаты-стемминга" data-toc-modified-id="Результаты-стемминга-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>Результаты стемминга</a></span></li></ul></div>

#### Загрузка библиотек и начальные операции

In [ ]:
import pandas as pd
import re
import numpy as np

import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')

from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

from nltk.util import ngrams

from collections import Counter 

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Объявим языки для стемминга
snowball_rus = SnowballStemmer('russian')
snowball_eng = SnowballStemmer('english')

# Объявим переменные для очистки от ненужных слов
stop_words_rus = set(stopwords.words("russian"))
stop_words_eng = set(stopwords.words("english"))

In [ ]:
# Прижелании берем ограничение на ширину количество знаков в столбцах таблиц
# pd.set_option('display.max_colwidth', 0)  # Возврат к нормальному отображению - значение "80"

#### Загрузка начальных данных 

In [ ]:
# Загрузка начальных данных из файла, содержащего необработанные данные
path = 'C:\\Users\\Downloads\\'        # Путь к файлу 
initial_data = 'data_jobs.csv'      # Файл для тестирования
df = pd.read_csv(path + initial_data)

In [ ]:
# Просмотр результата загрузки файла:
df.head(2)

#### Подготовка данных

In [ ]:
# Приведем написание в столбце к нижнему регистру
df['vacancy_name'] = df['vacancy_name'].str.lower()
# Или:
# df['vacancy_name'] = map(str.lower, df['vacancy_name'])

In [ ]:
# Приведем написание в столбце к нижнему регистру
df['vacancy_description'] = df['vacancy_description'].str.lower()

In [ ]:
# Приведем написание в столбце к нижнему регистру
df['list_of_skill_names'] = df['list_of_skill_names'].str.lower()

In [ ]:
# Преобразуем данные в формат даты из текстового формата
df['publication_day'] = pd.to_datetime(df['publication_day'])

In [ ]:
# Удалим столбец с непонятными значениями
df.drop('Unnamed: 0', inplace=True, axis='columns')

In [ ]:
# Проверим результат
df.head(2)

#### ВНИМАНИЕ: ключевой раздел! Создание выборки по ключевым словам

In [ ]:
# Выберем стороки, содержащие в "vacancy_name", например, слова "qa | q/a | тестировщик | quality"
# ВНИМАНИЕ: все буквы ключевых слов вводим строчными буквами (в ниженем регистре)

jobs_qa = df.query('vacancy_name.str.contains("аналитик данных | data analyst | DA ")', engine='python').reset_index().copy()

# Сохраним отфильтрованные данные для истории в файле:
jobs_qa.to_csv('jobs_qa.csv')

#### Загрузка отфильтрованных данных (короткий вариант, если фильтрация по словам уже выполнена)

In [ ]:
# # Это вариант загрузки данных для исследования напрямую из загружаемого файла "jobs_qa.csv"
# path = 'C:\\Users\\.......\\'  # Необходимо полностью указать путь к файлу
# file = 'jobs_qa.csv'
# jobs_qa = pd.read_csv(path + file)

#### Исследование данных

In [ ]:
jobs_qa.info()

In [ ]:
# Проверим результат:
jobs_qa.tail()

#### Очистка от дубликатов

In [ ]:
# Выполним поиск дубликатов и их удаление
print("Кол-во дубликатов до очистки:", jobs_qa.duplicated().sum())
print("Кол-во дубликатов в определенных столбцах до очистки:", jobs_qa.duplicated(['vacancy_name', 'vacancy_description', 'list_of_skill_names', 'city_name']).sum())
jobs_qa = jobs_qa.drop_duplicates(['vacancy_name', 'vacancy_description', 'list_of_skill_names', 'city_name'])
print("Кол-во дубликатов после очистки:", jobs_qa.duplicated(['vacancy_name', 'vacancy_description', 'list_of_skill_names', 'city_name']).sum())

#### Формирование данных с требованиями к кандидатам

In [ ]:
# Сформируем два Series с данным столбцов со списком требований к кандидатам и описанием вакансии:
skills = jobs_qa['list_of_skill_names']
vacancy_description = jobs_qa['vacancy_description']

In [ ]:
# Подставляем поочереди в строку ниже слова "обязанности", "навыки", "требования", "условия"
# и выделяем строки, содержащие в заголовке инфо, подставляя в поиск нужное слово из списка выше:
# ВНИМАНИЕ: какой Series подставим ниже ("skills" или "vacancy_description") - по тому и будет производиться выборка!
data_splitted = vacancy_description.str.findall(r'<strong>\w*требован\w*:<\/strong>(.+?)<strong>')

In [ ]:
# Произведем преабразования, необходимые в дальнейшем
data_splitted_explode = data_splitted.explode()
data_splitted_explode = data_splitted_explode.dropna()

In [ ]:
# Функция обработки слов с строках:

def data_to_string(text):
    # Разделим слова и словосочетания по запятой
    data_splited = text.str.split(',')

    # Преобразуем список списков в простой список
    data_splited = sum(data_splited, [])

    return data_splited

#### Очистка данных: удаление ненужных знаков и символов

In [ ]:
# Функция очистки данных
def preprocess(text):
    clean_data = []
    for x in (text[:]):                             # Альтернативный вариант: for x in (text[:][0])
        new_text = re.sub('<.*?>', '', x)           # remove HTML tags
        new_text = re.sub(r'[^\w\s]', '', new_text) # remove punc.
        new_text = re.sub(r'\d+','',new_text)       # remove numbers
        # new_text = new_text.lower()                 # lower case, .upper() for upper
        new_text = re.sub(r'none', '', new_text)    # remove "none"
        new_text = re.sub(r'qa', '', new_text)      # remove "qa"
        new_text = re.sub(r'\"', '', new_text)      # remove "
        new_text = re.sub(r'^\s+|\s\s+$', '', new_text) # Удаление пробелов в начале и конце фразы
        new_text = re.sub(r'^\s\s+', '', new_text)  # Удаление пробелов в начале и конце фразы
        new_text = re.sub(r'li', '', new_text)      # remove "li"
        new_text = re.sub(r'ul', '', new_text)      # remove "ul"
        new_text = re.sub(r'^p$', '', new_text)     # remove одиночное "p"
        
        if new_text != '':
            clean_data.append(new_text)
    return clean_data

#### Подсчет количества повторяющихся слов 

In [ ]:
# Функция очистки и подсчета количества повторяющихся слов
def count_words(text):
    
    # Очистка от предлогов и проч. ненужных и вспомогательных слов
    without_stop_words = [word for word in text if not word in stop_words_rus] 
    data_cleared = [word for word in without_stop_words if not word in stop_words_eng] 
    
    # Подсчитаем количество повторяющихся слов и фраз и сохраним результат в переменной
    phrase_count = Counter(data_cleared)

    # Выведем на печать наиболее часто встречающихся 100 словосочетаний
    top_phrase_list = phrase_count.most_common(100)
    top_phrase_list
    
    # Преобразуем в DataFrame
    top_words = pd.DataFrame(columns=['Ключевые_слова', 'Количество'], data=top_phrase_list)

    # Удалим строку с пробелом в значении
    top_words = top_words.query('Ключевые_слова != " "')

    # Удалим двойные пробелы между словами
    top_words = top_words.replace(r'\s\s','',regex=True)

    # Сгруппируем данные повторно, сохраним результат в DataFrame и выведем на экран:
    top_words_groupped = top_words.groupby('Ключевые_слова')['Количество'].sum().sort_values(ascending=False)
    top_words_groupped_df = pd.DataFrame(columns=['Количество'], data=top_words_groupped)

    # Уберем одиночное слово "тестирование"
    top_words_groupped_df = top_words_groupped_df.query('Ключевые_слова != "тестирование"')

    top_words_groupped_df = top_words_groupped_df.reset_index()
    print('Наиболее часто встречающиеся слова:')
    display(top_words_groupped_df.head(20))
    return(top_words_groupped_df)

In [ ]:
# Построение гистограммы
def hist_plot(text):
    
    fig = go.Figure(go.Bar(x=text['Ключевые_слова'].head(20),
                           y=text['Количество'],
                           text=text['Количество'].round(0),
                           textposition='outside',
                           textfont_size=12,
                           marker_color= 'DarkSlateGrey'
                           )
                   )

    fig.update_layout(title='ТОП-20 ключевых слов и фраз',
                      title_font_size=14,
                      height=700,
                      autosize=False,
                      width=1000,
                      plot_bgcolor='#e2eeee',
                      yaxis_title='Количество повторений',
                      xaxis_tickangle=-40,
                      xaxis_categoryorder= "total descending"
                     )
    fig.show()

#### Выделение единичных уникальных слов (токенизация по словам)

In [ ]:
# Функция выделения единичных уникальных слов (токенизации по словам)
def token():
    skills_cleared_str = str(data_cleared)
    
    # Выделим слова и сохраним в переменной "slkills_tokenizeds" 
    skills_tokenizeds = []
    for sentence in data_cleared:
        skills_tokenizeds.append(nltk.word_tokenize(sentence))
    
    # Склеим в единый список полученные слова
    skills_melted = []
    for i in skills_tokenizeds:
        skills_melted.extend(i)
    
    # Очистим от предлогов и вспомогательных слов
    stop_words_rus = set(stopwords.words("russian"))
    stop_words_eng = set(stopwords.words("english"))

    skills_without_stop_words_ru = [word for word in skills_melted if not word in stop_words_rus] 
    skills_without_stop_words = [word for word in skills_without_stop_words_ru if not word in stop_words_eng]
    
    # Подсчитаем количество единичных уникальных слов (без использования стемминга)
    skills_count_words_0 = Counter(skills_without_stop_words)
    
    # Выведем на экран первые n-слов (без использования стемминга)
    skills_top = skills_count_words_0.most_common(30)
    
    # Преобразуем данные в DataFrame и выведем результат на экран
    skills_top_df = pd.DataFrame(columns=['Ключевые_слова', 'Количество'], data=skills_top)

    print('Выделение единичных уникальных слов (токенизация по словам):')
    display(skills_top_df.head(20))
    
    # Построение гистограммы распределения:
    fig = go.Figure(go.Bar(x=skills_top_df['Ключевые_слова'].head(20),
                           y=skills_top_df['Количество'],
                           text=skills_top_df['Количество'].round(0),
                           textposition='outside',
                           textfont_size=12,
                           marker_color= 'DarkSlateGrey'
                           )
                   )

    fig.update_layout(title='ТОП-20 ключевых слов и фраз',
                      title_font_size=14,
                      height=600,
                      autosize=False,
                      width=1000,
                      plot_bgcolor='#e2eeee',
                      yaxis_title='Количество слов и фраз',
                      xaxis_tickangle=-45,
                      xaxis_categoryorder= "total descending"
                     )
    fig.show()

#### Нахождение словосочетаний (n-grams)

In [ ]:
# Функция нахождения словосочетаний (n-grams)
def n_grams_func():
    # Преобразуем данные в строковый формат
    data_cleared_str = str(data_splited)
    
    # Функция n-grams
    def get_ngrams(text, n):
        n_grams = ngrams(word_tokenize(text), n)
        return [' '.join(grams) for grams in n_grams]
    
    # Определим количество слов в словосочетании
    n = 6     

    # Запустим функцию нахождения n-grams
    ngrams_result = get_ngrams(data_cleared_str, n)
    
    # Повторно очистим текст от ненужных знаков (пунктуации)
    ngrams_result = preprocess(ngrams_result)
    
    # Подсчитаем результат и найдем самые часто употребляемые слова и словосочетания
    count_grams = Counter(ngrams_result)
    count_grams = count_grams.most_common(100)

    # Преобразуем полученные данные в DataFrame 
    skills_top_df = pd.DataFrame(columns=['Ключевые_слова', 'Количество'], data=count_grams)

    # Удалим двойные пробелы между словами
    # skills_top_df = skills_top_df.replace('^\s+|\s\s+$', '',regex=True)

    # Сгруппируем данные повторно, сохраним результат в DataFrame и выведем на экран:
    n_grams_top_groupped = skills_top_df.groupby('Ключевые_слова', as_index=False)['Количество'].sum()
    n_grams_top_groupped = n_grams_top_groupped.sort_values(by='Количество', ascending=False).reset_index(drop=True)

    # Уберем одиночное слово "тестирование"
    n_grams_top_groupped = n_grams_top_groupped.query('Ключевые_слова != "тестирование"')
    
    # Выведем результат на экран
    print('Словосочетания (n-grams), n=', n)
    display(n_grams_top_groupped.head(20))
    
    # Построение гистограммы распределения:
    fig = go.Figure(go.Bar(x=n_grams_top_groupped['Ключевые_слова'].head(20),
                           y=n_grams_top_groupped['Количество'],
                           text=n_grams_top_groupped['Количество'].round(0),
                           textposition='outside',
                           textfont_size=12,
                           marker_color= 'DarkSlateGrey'
                           )
                   )

    fig.update_layout(title='ТОП-20 ключевых слов и фраз',
                      title_font_size=14,
                      height=600,
                      autosize=False,
                      width=1000,
                      plot_bgcolor='#e2eeee',
                      yaxis_title='Количество слов и фраз',
                      xaxis_tickangle=-45,
                      xaxis_categoryorder= "total descending"
                     )
    fig.show()

#### Вывод результата на экран

In [ ]:
# Запустим выполнение функций: 
data_splited = data_to_string(data_splitted_explode)
data_cleared = preprocess(data_splited)
top_words_groupped_df = count_words(data_cleared )
hist_plot(top_words_groupped_df)

In [ ]:
token()

In [ ]:
n_grams_func()

#### Результаты стемминга

In [ ]:
# Функция стемминга (Russian and English)
def stemming(words):
    # Очистка от предлогов и проч. ненужных и вспомогательных слов
    without_stop_words = [word for word in words if not word in stop_words_rus] 
    data_cleared = [word for word in without_stop_words if not word in stop_words_eng] 
    
    new = []
    
    
    stem_words_rus = [snowball_rus.stem(x) for x in data_cleared]
    stem_words = [snowball_eng.stem(x) for x in stem_words_rus]
    new.append(stem_words)
    return new

In [ ]:
# Выполним функцию стемминга и сохраним результат в переменной
skills_without_stop_words_stem = stemming(data_cleared)

In [ ]:
# Преобразуем получившийся список списков в простой список
skills_without_stop_words_stem = sum(skills_without_stop_words_stem, [])

In [ ]:
# Подсчитаем количество уникальных слов в списке 
skills_count_words = Counter(skills_without_stop_words_stem)

In [ ]:
# Выведем на экран первые n-слов слов списка ранжировав их по убыванию
skills_count_words = skills_count_words.most_common(20)

In [ ]:
# Преобразуем данные в DataFrame и выведем результат на экран
skills_top_stemming = pd.DataFrame(columns=['Ключевые_слова', 'Количество'], data=skills_count_words)
print('Результаты стемминга:' )
skills_top_stemming.head(20)